# DHTML
- dynamic html
- 개발자도구의 Network 탭을 보면, Doc은 처음에 한번만 받아오고 그 다음에 검색해도 Doc을 새로 받아오지 않는다.
    - dom을 만들어도, 처음에 만든 dom에서 업데이트되지 않는다.
- HTML은 템플릿으로 두고, 검색할때 json을 받아서 내용을 업데이트한다.

In [37]:
import requests
from bs4 import BeautifulSoup
import re

# 연휴 끝난 기념, 몸풀기 문제
- http://example.webscraping.com/places/default/search
- 검색 결과 스크래핑하기
- 지난 시간까지 배웠던 것들을 다 기억하고 있으면 금방 풀 수 있을 것이다.

In [4]:
# 개발자도구의 Network 탭에서 필요한 url을 알 수 있다.

url = "http://example.webscraping.com/places/ajax/search.json"
params = {"search_term":"korea",
          "page_size":"10",
          "page":"0"}
resp = requests.request("get", url, params=params)

In [28]:
# json 파일을 받아왔다.

resp.headers["Content-type"]

'application/json'

> json 파일이므로 dom을 만들 필요가 없다.

In [30]:
# .json()메서드로 딕셔너리처럼 이용할 수 있다.

resp.json()

{'records': [{'pretty_link': '<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>',
   'country': 'North Korea',
   'id': 4908369},
  {'pretty_link': '<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>',
   'country': 'South Korea',
   'id': 4908415}],
 'num_pages': 1,
 'error': ''}

In [29]:
# 검색결과에서 국가 이름만 뽑아보자.

for _ in resp.json()["records"]:
    print(_["country"])

North Korea
South Korea


In [57]:
# 검색결과에서 여러가지 정보들을 뽑아보자.

for _ in resp.json()["records"]:
    print(_["country"])  # 국가명
    print(_["pretty_link"])  # html 코드 정보
    print(re.search(r"<img src=\"(.+)\" \/>",
                   _["pretty_link"]).groups())  # 이미지 링크
    print(requests.compat.urljoin(resp.url,
                                 re.findall(r"<img src=\"(.+)\" \/>", _["pretty_link"])[0])) # 이미지 링크 full url
    url = requests.compat.urljoin(resp.url,
                                 re.findall(r"<a href=\"(.+)\">", _["pretty_link"])[0]) 
    dom = BeautifulSoup(requests.request("get", url).text, "lxml")
    print(dom.form.text) # 검색 결과의 내용물

North Korea
<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>
('/places/static/images/flags/kp.png',)
http://example.webscraping.com/places/static/images/flags/kp.png
National Flag: Area: 120,540 square kilometresPopulation: 22,912,177Iso: KPCountry: North KoreaCapital: PyongyangContinent: ASTld: .kpCurrency Code: KPWCurrency Name: WonPhone: 850Postal Code Format: ###-###Postal Code Regex: ^(\d{6})$Languages: ko-KPNeighbours: CN KR RU 
South Korea
<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>
('/places/static/images/flags/kr.png',)
http://example.webscraping.com/places/static/images/flags/kr.png
National Flag: Area: 98,480 square kilometresPopulation: 48,422,644Iso: KRCountry: South KoreaCapital: SeoulContinent: ASTld: .krCurrency Code: KRWCurrency Name: WonPhone: 82Postal Code Format: SEOUL ###-###Postal Code Regex: ^(?:SEOUL)*(\d{6})$La

---

### 다른 방법

In [15]:
# 검색결과에서 이미지 소스만 뽑아보자.

js["records"][0]["pretty_link"]

'<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>'

> img src만 가져오기 위해서 정규식을 이용한다.

In [17]:
re.findall(r'img src="(.*?)"', js["records"][0]["pretty_link"])

['/places/static/images/flags/kp.png']

In [33]:
# list comprehension으로 한번에 가져오기.
# 

imgs = [requests.compat.urljoin(resp.url,
                                re.findall(r'img src="(.*?)"', _["pretty_link"])[0]) 
        for _ in js["records"]]
imgs

['http://example.webscraping.com/places/static/images/flags/kp.png',
 'http://example.webscraping.com/places/static/images/flags/kr.png']